# phase 1

data exploration and cleaning

## data exploration

In [3]:
import pandas as pd

In [ ]:
chunks = pd.read_json("yelp_academic_dataset_review.json", lines=True, chunksize=100000)

for chunk in chunks:
    print("HEAD")
    print(chunk.head())
    print("\nCOLUMNS")
    print(chunk.columns)
    break

print("\nINFO")
for chunk in chunks:
    print(chunk.info())

HEAD
                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
2  saUsX_uimxRlCVr67Z4Jig  8g_iMtfSiwikVnbP2etR0A  YjUWPpI6HXG530lwP-fb2A   
3  AqPFMleE6RsU23_auESxiA  _7bHUi9Uuf5__HHc_Q8guQ  kxX2SOes4o-D3ZQBkiMRfA   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   

   stars  useful  funny  cool  \
0      3       0      0     0   
1      5       1      0     1   
2      3       0      0     0   
3      5       1      0     1   
4      4       1      0     1   

                                                text                date  
0  If you decide to eat here, just be aware it is... 2018-07-07 22:09:11  
1  I've taken a lot of spin classes over the year... 2012-01-03 15:28:18  
2  Family diner. Had the buffet. Eclectic assortm... 2014-02-05 20:30:30  
3  Wow!  Yummy, different,  delic

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000021DD9E712B0>>
Traceback (most recent call last):
  File "c:\Users\Lenovo\myFiles\nauka\studia\sem6\uczenie maszynowe\laby\sentiment_analysis\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 790, in _clean_thread_parent_frames
    active_threads = {thread.ident for thread in threading.enumerate()}
  File "C:\Users\Lenovo\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1477, in enumerate
    def enumerate():
KeyboardInterrupt: 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 900000 to 999999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   review_id    100000 non-null  object        
 1   user_id      100000 non-null  object        
 2   business_id  100000 non-null  object        
 3   stars        100000 non-null  int64         
 4   useful       100000 non-null  int64         
 5   funny        100000 non-null  int64         
 6   cool         100000 non-null  int64         
 7   text         100000 non-null  object        
 8   date         100000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 6.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 1000000 to 1099999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   review_id    100000 non-null  object   

## data cleaning

In [ ]:
# dropping irrelevant columns

irrelevant_columns = ['review_id', 'user_id', 'business_id', 'useful', 'funny', 'cool', 'date']
output_file = "yelp_reviews_dropped_columns.csv"

for i, chunk in enumerate(chunks):
    chunk = chunk.drop(columns=irrelevant_columns)
    if i == 0:
        print(chunk.columns)
    chunk.to_csv(output_file, mode="a", index=False, header=(i == 0))

In [ ]:
csv_chunks = pd.read_csv("yelp_reviews_dropped_columns.csv", chunksize=100000)

for chunk in csv_chunks:
    print("HEAD")
    print(chunk.head())
    print("\nCOLUMNS")
    print(chunk.columns)
    break

HEAD
   stars                                               text
0      3  Why 3 stars?\n\nI love Target, but here is jus...
1      4  The Local Taco is the first joint I've found i...
2      5  Top notch! He will cater our wedding and  Fran...
3      5  Best place to get a vietnamese sandwich and bo...
4      4  This is more like a 3 1/2 star rating. The foo...

COLUMNS
Index(['stars', 'text'], dtype='object')
